In [1]:
#!pip install pandas
#!pip install requests

In [5]:
import pandas as pd
import requests

In [6]:
text_file = 'dests.txt'
df = pd.DataFrame(columns=['Target', 'Distance_km', 'Duration', 'Longitude', 'Latitude'])

api_key = ' ' #my API code - I removed it before i uploaded it to github'

with open(text_file, 'r') as file:
    for destination in file:
        destination = destination.strip()

        #distance matrix - get the distance between the city of Tel Aviv and the destination in kilometers
        distance_matrix = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=Tel+Aviv&destinations=' + destination + '&key=' + api_key
        
        #geocode - The longitude and latitude of the target 
        geocode = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + destination + '&key=' + api_key

        try:
            
            distance_data = requests.get(distance_matrix).json()
            distance = distance_data['rows'][0]['elements'][0]['distance']['text']
            
            #The time it takes to reach the destination in minutes
            duration = distance_data['rows'][0]['elements'][0]['duration']['text']

            geocode_data = requests.get(geocode).json()
            longitude = geocode_data['results'][0]['geometry']['location']['lng']
            latitude = geocode_data['results'][0]['geometry']['location']['lat']

            #converts the distance from a string to a number
            distance = float(distance.replace(',', '').replace(' km', ''))

            df = pd.concat(
                [df, pd.DataFrame({'Target': [destination],'Distance_km': [distance],
                'Duration': [duration],'Longitude': [longitude],'Latitude': [latitude]})
                ],ignore_index=True)

        except Exception as e:
            print("Error occurred for destination: {}".format(destination))
            print(e)

print(df)


      Target  Distance_km          Duration  Longitude   Latitude
0   Istanbul       1815.0   21 hours 3 mins  28.978359  41.008238
1  Amsterdam       4533.0    2 days 0 hours   4.904139  52.367573
2   Valletta       3793.0    2 days 3 hours  14.514100  35.899237
3      Basel       4093.0    1 day 20 hours   7.588576  47.559599
4       Doha       2164.0  22 hours 39 mins  51.531040  25.285447


In [7]:
furthest_cities = df.nlargest(3, 'Distance_km')
print(furthest_cities)

      Target  Distance_km        Duration  Longitude   Latitude
1  Amsterdam       4533.0  2 days 0 hours   4.904139  52.367573
3      Basel       4093.0  1 day 20 hours   7.588576  47.559599
2   Valletta       3793.0  2 days 3 hours  14.514100  35.899237
